In [1]:
import sys

sys.path.insert(0, '..')

In [9]:
import os
import numpy as np
import random
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction, TimeSeriesTransformerModel
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import ReduceLROnPlateau
from scipy import stats

from core.dataset import MachoDataset
from core.trainer import PredictionTrainer, ClassificationTrainer
from core.model import ClassificationModel

In [3]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True

In [4]:
config = {
    'random_seed': random_seed,
    'data_root': '/home/mrizhko/AML/contra_periodic/data/macho/',
    'balanced_data_root': '/home/mrizhko/AML/AstroML/data/macho-balanced/',
    'weights_path': '/home/mrizhko/AML/AstroML/weights/',

    # Time Series Transformer
    'lags': None,  # ?
    'distribution_output': 'normal',
    'num_static_real_features': 0,  # if 0 we don't use real features
    'num_time_features': 1,
    'd_model': 256,
    'decoder_layers': 4,
    'encoder_layers': 4,
    'dropout': 0,
    'encoder_layerdrop': 0,
    'decoder_layerdrop': 0,
    'attention_dropout': 0,
    'activation_dropout': 0,

    # Data
    'window_length': 200,
    'prediction_length': 10,  # 1 5 10 25 50

    # Training
    'batch_size': 512,
    'lr': 0.0001,
    'weight_decay': 0,
    'epochs_pre_training': 1000,
    'epochs_fine_tuning': 100,
    
    # Learning Rate Scheduler
    'factor': 0.3,
    'patience': 10,

    'mode': 'pre-training',  # 'pre-training' 'fine-tuning' 'both'
    'save_weights': False,
    'config_from_run': None,  # 'MeriDK/AstroML/qtun67bq'
}

In [5]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('Using', device)

In [6]:
train_dataset = MachoDataset(config['balanced_data_root'], config['prediction_length'], mode='train')
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)

In [7]:
transformer_config = TimeSeriesTransformerConfig(
    prediction_length=config['prediction_length'],
    context_length=config['window_length'] - config['prediction_length'] - 7,  # 7 is max(lags) for default lags
    num_time_features=config['num_time_features'],
    num_static_real_features=config['num_static_real_features'],
    encoder_layers=config['encoder_layers'],
    decoder_layers=config['decoder_layers'],
    d_model=config['d_model'],
    distribution_output='normal',
    scaling=None,
    dropout=config['dropout'],
    encoder_layerdrop=config['encoder_layerdrop'],
    decoder_layerdrop=config['decoder_layerdrop'],
    attention_dropout=config['attention_dropout'],
    activation_dropout=config['activation_dropout']
)

In [12]:
model = TimeSeriesTransformerForPrediction(transformer_config)
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=config['factor'], patience=config['patience'], verbose=True)

trainer = PredictionTrainer(model=model, optimizer=optimizer, scheduler=scheduler, device=device)

In [13]:
trainer.train(train_dataloader, train_dataloader, epochs=1000)

In [17]:
val_dataset = MachoDataset(config['balanced_data_root'], config['prediction_length'], mode='val')
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [18]:
trainer.evaluate(val_dataloader, val_dataset)

In [19]:
trainer.val_epoch(val_dataloader)

In [21]:
trainer.val_epoch(train_dataloader)